# Sacramento Homeless Management Information System Data Summary
Homeless Management Information System (HMIS) data captures client-level information about housing services and other homelessness prevention services provided those acutely experiencing homlessness or who are at risk of homelessness. These data are for the Sacramento County continuum of care and are current through fall of 2019. Data quality prior to 2015 is variable as the system used to collect information was modified in January 2014. The initial goals of this project are to present these data in an interactive web format to allow users to better understand trends in homelessness in our region. The following python code attempts to merge and summarize the data in preparation for visualization for the following goals:

1. Summarize and visualize the movement of clients on an annual basis--eventually this would ideally be sub-annual (i.e., quarterly/monthly basis).
2. Summarize and visualize the volume of clients served by various programs and their outcomes, particuarly what proportion of recorded exits are to temporary  or permanent housing 
3. Summarize and visualize volume and movement of clients in and out of programs broken down by demographic characteristics including, but not limited to gender, race/ethnicity, veteran status, age groups, etc. 

In [1]:
#read in libraries
import pandas as pd
import numpy as np
import math as math
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

#establish path to the data folder in the github repo
master_url = "https://raw.githubusercontent.com/code4sac/sacramento-county-homeless-hmis-data/master/data/"

#get the individual file names
assessment_file = "Sacramento_County_-_Assessment_Table_2019-09-05T0401_pTq3TT.csv"
client_file = "Sacramento_County_-_Client_Table_2019-09-05T0101_Kky8n7.csv"
enrollment_file = "Sacramento_County_-_Enrollment_Table_2019-09-05T0131_KptDcM.csv"
exit_file = "Sacramento_County_-_Exit_Table_2019-09-01T0601_FDwNWs.csv"
project_file ="Sacramento_County_-_Project_Table_2019-09-05T0200_DdZb5N.csv"
service_file = "Sacramento_County_-_Service_Table_2019-09-05T0301_HZ8K2P.csv"
    
#read in each csv file as a dataframe
assessment = pd.read_csv (master_url + assessment_file)
client = pd.read_csv (master_url + client_file)
enrollment = pd.read_csv (master_url + enrollment_file)
exit = pd.read_csv (master_url + exit_file)
project = pd.read_csv (master_url + project_file)
service = pd.read_csv (master_url + service_file)


C:\Users\ASW\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The following standardizes the formatting of the personal id (i.e., client-level) and enrollment id values

In [2]:
##clean up the id fields where necessary (some of them have commas and were therefore read in as string instead of a number)

assessment['Personal ID'] =pd.to_numeric(assessment['Personal ID'].str.replace(',', ''))
client['Personal ID'] = pd.to_numeric(client['Personal_Id_d'].str.replace(',',''))
enrollment['Enrollment Id'] =pd.to_numeric(enrollment['Enrollment Id'].str.replace(',', ''))
exit['Personal ID'] =pd.to_numeric(exit['Personal ID'].str.replace(',', ''))
exit['Enrollment Id'] =pd.to_numeric(exit['Enrollment Id'].str.replace(',', ''))


    

Based on conversation with a subject matter expert, specific fields of interest were subset from each data frame for inclusion in the final merged product. There are more fields that may be of interest if the scope of work expands. See this file for the metadata: "Variables included in County Data.xlsx"

In [3]:
##subset the columns of interest

#enrollment.head()
enrollment_subset = enrollment[['Personal ID', 'Household ID', 'Enrollment Id', 'Entry Screen Added Date',
                   'Enrollments Project Id','Entry Screen Housing Status',
                   'Entry Screen Chronic Homeless at Project Start',
                   'Entry Screen Residence Prior to Project Entry']]

#client.head()
client_subset = client[['Personal ID','Birth_Date_d','Clients Race'
                       ,'Clients Ethnicity','Clients Gender','Clients Veteran Status']]

#project.head()
program_subset = project[['Program Id', 'Agency Id','Name','Project Type Code']]

#exit.head()
exit_subset = exit[['Enrollment Id','Exit Destination','Exit Reason'
                   ,'Project Exit Date']]

In [4]:
##join the data frames together

en_cl = pd.merge(enrollment_subset, client_subset,on='Personal ID', how = 'left')

en_cl_pg = pd.merge(en_cl, program_subset, left_on = 'Enrollments Project Id', right_on = 'Program Id',how = 'left')

en_cl_pg_ex = pd.merge(en_cl_pg, exit_subset, on = 'Enrollment Id',how = 'left')


Some additional fields of interest are calculated by the following code
1. Entry year and exit year, derived from the entry date and exit date
2. Whether the enrollment was ongoing at the start of each calendar year. This has to be created for each year of interest because it's possible for enrollments to span multipley years. For example a client could enroll in December 2015 and exit in January 2017. In this case their enrollment would be active at the start of 2016 and 2017 although they neither exited nor entered in 2016.
3. The duration of the enrollment (i.e., number of days between entry and exit)
4. How old the client was at the time of entry
5. What broader exit destination group the enrollment resulted in. For the look up table see the worksheet "Exit Destination" worksheet in the "Variables included in County Data.xlsx" workbook. 

In [5]:
##calculate some additional fields of interest 

#convert the columns that contain date values from strings to a date data type
en_cl_pg_ex['Entry Screen Added Date'] = pd.to_datetime(en_cl_pg_ex['Entry Screen Added Date'], errors = 'coerce')
en_cl_pg_ex['Project Exit Date'] = pd.to_datetime(en_cl_pg_ex['Project Exit Date'], errors = 'coerce')
en_cl_pg_ex['Birth_Date_d'] = pd.to_datetime(en_cl_pg_ex['Birth_Date_d'], errors = 'coerce')

#derive the entry and exit years
en_cl_pg_ex['entry_year']=  pd.DatetimeIndex(en_cl_pg_ex['Entry Screen Added Date']).year
en_cl_pg_ex['exit_year']=  pd.DatetimeIndex(en_cl_pg_ex['Project Exit Date']).year

#flag if the enrollment was present at the start of each calendar year (this will be used to tally starting numbers for each year)
en_cl_pg_ex['present_2015'] = np.where((en_cl_pg_ex.entry_year < 2015) & ((en_cl_pg_ex.exit_year >= 2015) | (en_cl_pg_ex.exit_year.isnull())) , 1, 0)
en_cl_pg_ex['present_2016'] = np.where((en_cl_pg_ex.entry_year < 2016) & ((en_cl_pg_ex.exit_year >= 2016) | (en_cl_pg_ex.exit_year.isnull())) , 1, 0)
en_cl_pg_ex['present_2017'] = np.where((en_cl_pg_ex.entry_year < 2017) & ((en_cl_pg_ex.exit_year >= 2017) | (en_cl_pg_ex.exit_year.isnull())) , 1, 0)
en_cl_pg_ex['present_2018'] = np.where((en_cl_pg_ex.entry_year < 2018) & ((en_cl_pg_ex.exit_year >= 2018) | (en_cl_pg_ex.exit_year.isnull())) , 1, 0)
en_cl_pg_ex['present_2019'] = np.where((en_cl_pg_ex.entry_year < 2019) & ((en_cl_pg_ex.exit_year >= 2019) | (en_cl_pg_ex.exit_year.isnull())) , 1, 0)

#calculate the duration of each enrollment (i.e., exit date - entry date) in days
en_cl_pg_ex['enrollment_duration'] = round((en_cl_pg_ex['Project Exit Date'] - en_cl_pg_ex['Entry Screen Added Date'])/np.timedelta64(1,'D'),1)

#generate the client age at enrollment--first convert the dates stored as string to datetime values the get the difference
en_cl_pg_ex['age_at_entry'] = round((en_cl_pg_ex['Entry Screen Added Date'] - en_cl_pg_ex['Birth_Date_d'])/np.timedelta64(1,'Y'),1)

#create age brackets of interest
age_brackets = [
    (en_cl_pg_ex['age_at_entry'] <18),
    (en_cl_pg_ex['age_at_entry'] <= 25) ,
    (en_cl_pg_ex['age_at_entry'] > 25) & (en_cl_pg_ex['age_at_entry'] < 65),
    (en_cl_pg_ex['age_at_entry'] >=65)
]
age_groups = ['Child','Young Adult','Adult','Senior']

en_cl_pg_ex['age_group'] = np.select(age_brackets, age_groups, default='null')


#group the exit destinations into more succint categories (ask Chris where he got these values)
#make a copy of the exit destination field 
en_cl_pg_ex['exit_destination_code'] = en_cl_pg_ex['Exit Destination']

exit_code_dictionary = {"Client doesn't know": 4,
'Client refused': 4,
'Data not collected': 4,
'Deceased': 5,
'Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter': 2,
'Foster care home or foster care group home': 6,
'Hospital or other residential non-psychiatric medical facility': 6,
'Hotel or motel paid for without emergency shelter voucher': 2,
'Jail, prison or juvenile detention facility': 6,
'Long-term care facility or nursing home': 6,
'Moved from one HOPWA  funded project to HOPWA TH': 1,
'Moved from one HOPWA funded project to HOPWA PH': 2,
'No exit interview completed': 4,
'Other': 4,
'Owned by client, no ongoing housing subsidy': 1,
'Owned by client, with ongoing housing subsidy': 1,
'Permanent housing (other than RRH) for formerly homeless persons': 1,
'Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)': 3,
'Psychiatric hospital or other psychiatric facility': 6,
'Rental by client, no ongoing housing subsidy': 1,
'Rental by client, with GPD TIP housing subsidy': 1,
'Rental by client, with other ongoing housing subsidy': 1,
'Rental by client, with RRH or equivalent subsidy': 1,
'Rental by client, with VASH housing subsidy': 1,
'Residential project or halfway house with no homeless criteria': 1,
'Safe Haven': 2,
'Staying or living with family, permanent tenure': 1,
'Staying or living with family, temporary tenure (e.g. room, apartment or house)': 2,
'Staying or living with friends, permanent tenure': 1,
'Staying or living with friends, temporary tenure (e.g. room, apartment or house)': 2,
'Substance abuse treatment facility or detox center': 6,
'Transitional housing for homeless persons (including homeless youth)': 2,
'NaN': 0
}
en_cl_pg_ex= en_cl_pg_ex.replace({'exit_destination_code': exit_code_dictionary})

#convert these numeric codes into groups with names
en_cl_pg_ex['exit_destination_group'] = en_cl_pg_ex['exit_destination_code']
exit_group_dictionary ={1:'Permanent', 2: 'Temporary', 3: 'Place not meant for human habitation', 4:'Unknown',
                        5:'Deceased', 6:'Institutionalized', 0:'No Data - Currently Enrolled'
}

en_cl_pg_ex= en_cl_pg_ex.replace({'exit_destination_group': exit_group_dictionary})


My understanding is that the data system/vendor was switched in 2014, so data quality prior to 2015 can be problematic. Based on conversation with a subject matter expert the data prior to that should probably not be included in analysis. Deciding exactly which date to use to truncate the information, however, should be confirmed. 

In [6]:
#subset the data to include only that captured after 1/1/2015 per advice from Chris
en_cl_pg_ex_2015 = en_cl_pg_ex [en_cl_pg_ex['Entry Screen Added Date'] >= '1/1/2015']


To generate the highest overview picture of client inflow and outflow on an annual basis the needed figures are the number of currently enrolled clients + the number of new clients - the number of clients that  exited to temporary or permanent housing. 

The code below attempts to tally the annual number of unique clients that entered each year, the number of unique clients that exited to temporary or permanent housing each year, and the number of clients that started out the year being enrolled in a program. **What's below is incorrect and needs revision as it does not account for clients that had multiple enrollments in a single year, particularly if the enrollments had different outcomes. Advice from a subject matter expert would also be very useful to evaluate how often clients are enrolled in multiple programs at the same time, and how to handle mixed outcomes (i.e., should the most recent outcome be counted, or the most common outcome be counted, etc.). Because data are truncated prior to 2015, it also fails to capture the starting level of engagement in 2015 and therefore the logic in the previous step might need to be reconsidered.**

In [7]:
##summarize the clients presently enrolled, newly enrolled, and exited to housing for each year

#tally how many unique clients were presently enrolled in a program at the start of each calendar year
##NOTE THIS IS NOT CALCULATED CORRECTLY!--needs to be revised to pull distinct personal id's rather than sum all enrollments
annual_present = en_cl_pg_ex_2015[['present_2015','present_2016','present_2017','present_2018','present_2019']].sum()
annual_present = pd.DataFrame({'Year_Present':annual_present.index, 'Initial_Count':annual_present.values})
annual_present['Year']= annual_present['Year_Present'].str.replace('present_','')
annual_present['Year'] = annual_present['Year'].astype(int)
annual_present = annual_present[['Year','Initial_Count']]

#tally how many unique clients enrolled each year
annual_entry = en_cl_pg_ex_2015[['Personal ID', 'entry_year']].groupby(['entry_year']).agg(['nunique'])
annual_entry = annual_entry.reset_index()
annual_entry.columns = ['Year','Enrollment_Count']

#tally how many unique clients exited for permanent (1) or temporary (2) destinations each year
#subset only records where there was a temporary  or permanent housing exit outcome--ask Chris about this
en_cl_pg_ex_2015_exit = en_cl_pg_ex_2015[(en_cl_pg_ex_2015.exit_destination_code == 1) | (en_cl_pg_ex_2015.exit_destination_code == 2)]
annual_exit = en_cl_pg_ex_2015_exit[['Personal ID', 'exit_year']].groupby(['exit_year']).agg(['nunique'])
annual_exit = annual_exit.reset_index()
annual_exit.columns = ['Year','Exit_Count']


#combine these data frames
df1 = pd.merge(annual_present, annual_entry,how = 'left')
df2 = pd.merge(df1,annual_exit ,how = 'left')

#export the data to a csv file
df2.to_csv('annual_summary.csv') 
print(df2)




   Year  Initial_Count  Enrollment_Count  Exit_Count
0  2015              0              7039        3430
1  2016           3605              8489        4704
2  2017           6060              8920        4728
3  2018           6053              9613        5281
4  2019           6818              7243        3424


This groups the data by the program. It should produce the number of unique clients served, the number of total enrollments (which is different because one clients can receive services multiple times), as well as the exit destination outcomes of those enrollments. There are cases where there is no exit destination associated with an enrollment, presumably because the client is still currently. This assumption should be confirmed with a subject matter expert and closer inspection of the exit data itself--the code below does not have a category of 'Currently Enrolled' as an enrollment destination outcome and those are simply not tallied. 

In [8]:
##summarize enrollment/client volume and outcomes by the program

#tally the number of enrollments and unique clients by program name
overview = en_cl_pg_ex_2015[['Name', 'Personal ID', 'Enrollment Id']].groupby(['Name']).agg(['nunique'])

#tally the total, mean, and median duration (in days) of enrollment by program name
duration = en_cl_pg_ex_2015[['Name', 'enrollment_duration']].groupby(['Name']).agg(['sum','mean','median'])

#calculate the number of outcomes that fall into the six different categories for each program name
destination_counts = en_cl_pg_ex_2015.groupby(['Name', 'exit_destination_group']).size().reset_index(name='counts')
destination_counts = destination_counts.pivot(index='Name', columns='exit_destination_group', values='counts')


#merge data frames together and rename the columns
df3 = pd.merge(overview, duration, left_on = 'Name', right_on = 'Name',how = 'left')
df4 =  pd.merge(df3, destination_counts, left_on = 'Name', right_on = 'Name',how = 'left')
df4.columns = [ 'Client_Count','Enrollment_Count','Total_Enrollment_Duration','Avg_Enrollment_Duration'
              ,'Median_Enrollment_Duration','Deceased','Institutionalized','Permanent'
              ,'Place not meant for human habitation','Temporary','Unknown']

#export the data to a csv file
df4.to_csv('program_summary.csv') 
print(df4)



                                             Client_Count  Enrollment_Count  \
Name                                                                          
Ardenaire Apartments - PSH                             29                29   
Audre's Emporium of New Tomorrows - TH                 19                19   
Audre's RHY - TH                                       32                33   
Bishop Gallegos Maternity Home Shelter - ES            88                88   
Bridges SLE - TH                                       24                24   
...                                                   ...               ...   
YWCA - PSH                                             12                12   
Yolo SSVF - Prevention                                  2                 2   
Yolo SSVF - RRH                                         8                 8   
Yolo- SSVF-RRH (Retired)                                4                 4   
Yolo-SSVF-Prevention (Retired)                      

C:\Users\ASW\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
